In [2]:
import pandas as pd
import numpy as np

In [3]:
sensor_name = 'R1'
corr_xx = pd.read_feather(f'../data/corr/XX/corr_XX-{sensor_name}.feather')
corr_xy = pd.read_feather(f'../data/corr/XY/corr_XY-{sensor_name}.feather')

In [4]:
corr_xy

,NO_conc,CH4_conc,H2S_conc,SO2_conc,HCOH_conc,CO_conc,H2_conc,NO2_conc,NH3_conc
R1_0,0.415469,-0.061622,-0.138076,-0.072065,-0.134868,-0.121708,-0.137949,0.073004,-0.116817
R1_1,0.415481,-0.061619,-0.138093,-0.072069,-0.134883,-0.121723,-0.137965,0.073136,-0.116822
R1_2,0.414911,-0.061614,-0.138048,-0.071948,-0.134822,-0.121655,-0.137912,0.072828,-0.116754
R1_3,0.414659,-0.061586,-0.138042,-0.071845,-0.134798,-0.121606,-0.137897,0.072551,-0.116713
R1_4,0.413894,-0.061591,-0.137970,-0.071677,-0.134706,-0.121506,-0.137814,0.072063,-0.116608
...,...,...,...,...,...,...,...,...,...
R1_1196,0.412643,-0.062019,-0.138708,-0.071956,-0.135517,-0.122333,-0.138597,0.075028,-0.117368
R1_1197,0.412554,-0.062008,-0.138730,-0.071965,-0.135536,-0.122352,-0.138617,0.075245,-0.117373
R1_1198,0.412512,-0.061995,-0.138778,-0.071972,-0.135580,-0.122391,-0.138662,0.075471,-0.117398
R1_1199,0.412601,-0.061988,-0.138817,-0.071974,-0.135614,-0.122424,-0.138699,0.075694,-0.117415


In [7]:
def IFS(corXX, corXY, level_xx, level_xy):
    corXY = corXY.values
    mask = [False] * len(corXX)
    while np.max(corXY) > level_xy:
        i_bestXY = np.argmax(corXY)
        # print("Лучший:", i_bestXY, "\nПохожие:")
        mask[i_bestXY] = True
        corXY[i_bestXY] = 0
        for i in range(corXX.shape[0]):
            if mask[i] == False:
                if corXY[i]>0:
                    if corXX.iloc[i_bestXY, i]>level_xx:
                        corXY[i] = 0
                        # print(i)
    return mask

In [10]:
IFS(corr_xx, corr_xy['NO_conc'], 0.99, 0.0)

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,


In [23]:
my_l = pd.DataFrame([1,2,3])

In [24]:
def plus_1(df):
    df = df.copy()
    df[0][0] = 100
    return df

In [25]:
plus_1(my_l)

C:\Users\HONOR\AppData\Local\Temp\ipykernel_21352\2082112145.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df[0][0] = 100


,0
0,100
1,2
2,3


In [26]:
my_l

,0
0,1
1,2
2,3


In [16]:
my_l[0][0]

np.int64(1)